# Imports

In [18]:
import numpy as np, pandas as pd
import os, shutil
from distutils.dir_util import copy_tree

## Define source path for train and test data

In [19]:
'''
Define the path to the train and test data files on your local computer. 

src_path should be a directory which has two folders: 1) train, and 2) test
'train' folder must have a file called: ratings_train.csv
'test' folder must have a file called : ratings_test.csv 

Both 'ratings_train.csv' and 'ratings_text.csv' should have the following four fields (comma-separated): 
rating_id (string type)
user_id (string type)
item_id (string type)
rating (float)
'''


dataset = 'jester'      # jester, movielens-10m, movielens-20m, book-crossing

src_path = f'./data/{dataset}/processed/'

## Make Volume to Mount with appropriate folders
This is a folder created in this present directory. <br>
We will copy the data files into this folder and then bind mount this folder into the container. <br>
When the container is run, model artifacts and other outputs will also be saved to this same folder. <br>
These files on the shared volume will persist when the container is shut down. 

In [20]:
# Run this as-is. Do not change any folder names!!! 

mounted_volume = 'ml_vol'
if os.path.exists(mounted_volume): shutil.rmtree(mounted_volume)

os.mkdir(mounted_volume)
subdirs = ['data', 'logs', 'model', 'output']
for subdir in subdirs: 
    sub_dir_path = os.path.join(mounted_volume, subdir)
    os.mkdir(sub_dir_path)

subdirs = ['train', 'test']
for subdir in subdirs: 
    sub_dir_path = os.path.join(mounted_volume, 'data', subdir)
    os.mkdir(sub_dir_path)

## Copy Data from Source Path Into Mounted Volume

In [21]:
dest_path = os.path.join(mounted_volume, 'data')
for input_type in ['train', 'test']:
    full_src = os.path.join(src_path, input_type)
    full_dest = os.path.join(dest_path, input_type)
    
    if os.path.exists(full_src): copy_tree(full_src, full_dest)

# Build Image

In [22]:
%%bash 
docker build -t abudesai/rec_base_autorec:v1.0.0 ./autorec

#1 [internal] load build definition from Dockerfile
#1 sha256:cba5b5c785fae1ed092399f5d62f2d1f39e5430a1f8e51efacaa86b54df18dcb
#1 transferring dockerfile: 265B 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:ce9f53777f60918aa2a8289cb7251274f23af64c8464b040e79a5a4a9f8368d2
#2 transferring context: 62B 0.0s done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8.0-slim
#3 sha256:4f0c597550e30aa54f707f0017cf64d137017976c13b147baa6fd4ad0c55c91e
#3 ...

#4 [auth] library/python:pull token for registry-1.docker.io
#4 sha256:b3a7e066c5bf388af0a053fadb68adc4469e1474f94492aa8195ae738ff8706b
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8.0-slim
#3 sha256:4f0c597550e30aa54f707f0017cf64d137017976c13b147baa6fd4ad0c55c91e
#3 DONE 7.0s

#5 [1/5] FROM docker.io/library/python:3.8.0-slim@sha256:8e243f41e500238f78f7a29a81656114d3fe603d5c34079a462d090f71c4b225
#5 sha256:f2202870b184ece5b9a09c9b777f938cf0be25287ffe019e2c50e60191382ede


# Create Container From Image

In [23]:
%%bash 
declare vol_path="$(pwd)/ml_vol"
docker run -d -p 3000:3000 -v $vol_path:/app/ml_vol --name autorec abudesai/rec_base_autorec:v1.0.0

0fb3d84311f9168aadd2cb2f01fa7958549f533e93a8ba04df97f0b15897cfea


# Check Container and Image

In [24]:
%%bash 
docker ps

CONTAINER ID   IMAGE                              COMMAND                  CREATED         STATUS         PORTS                    NAMES
0fb3d84311f9   abudesai/rec_base_autorec:v1.0.0   "/bin/sh -c 'python â€¦"   4 seconds ago   Up 3 seconds   0.0.0.0:3000->3000/tcp   autorec


In [37]:
#%%bash 
#docker inspect mfc

# Run Training

In [26]:
%%bash 
docker exec autorec python train.py

Starting the training process...
orig_train_data shape: (3929542, 4)
After train/valid split, train_data shape: (3536374, 4) valid_data shape: (393168, 4)
Pre-processing data...
data based params: {'M': 100}
model_params: {'M': 100, 'drop_out': 0.7, 'lr': 0.08, 'momentum': 0.9}
Training matrix factorizer ...
Epoch 1/15
574/574 [==============================] - 7s 11ms/step - loss: 0.0587 - custom_loss: 0.0587 - val_loss: 0.0493 - val_custom_loss: 0.0493
Epoch 2/15
574/574 [==============================] - 6s 11ms/step - loss: 0.0471 - custom_loss: 0.0471 - val_loss: 0.0476 - val_custom_loss: 0.0476
Epoch 3/15
574/574 [==============================] - 6s 11ms/step - loss: 0.0457 - custom_loss: 0.0457 - val_loss: 0.0467 - val_custom_loss: 0.0467
Epoch 4/15
574/574 [==============================] - 6s 11ms/step - loss: 0.0450 - custom_loss: 0.0450 - val_loss: 0.0461 - val_custom_loss: 0.0461
Epoch 5/15
574/574 [==============================] - 6s 11ms/step - loss: 0.0445 - custom_los

2021-12-10 06:32:05.084691: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 06:32:05.084737: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-10 06:32:17.960068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-10 06:32:17.960110: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-10 06:32:17.960125: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0fb3d84311f9): /proc/driver/nvidia/version does not exist
2021-12-10 06:32:17.960268: I tensorflow/core/platform/cpu_featu

# Run Test Data Predictions

In [28]:
%%bash 
docker exec autorec python predict.py

Reading prediction data... 
Loading trained model... 
Preprocessing prediction data... 
Making predictions... 
Post-processing predictions for writing... 
Saving predictions... 
Done with predictions.


2021-12-10 06:34:36.414940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 06:34:36.414979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-10 06:34:50.392104: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-10 06:34:50.392143: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-10 06:34:50.392155: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0fb3d84311f9): /proc/driver/nvidia/version does not exist
2021-12-10 06:34:50.392290: I tensorflow/core/platform/cpu_featu

# Score Test Data Predictions

In [29]:
%%bash 
docker exec autorec python score.py

scores: {'mse': 17.453816578632445, 'rmse': 4.17777651133141, 'mae': 3.2842097552282747, 'nmae': 4.380761472706451, 'smape': 107.29605599119468, 'r2': 0.377373158133142}


2021-12-10 06:35:19.297206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 06:35:19.297252: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Check Outputs In mounted volume
- data: this is where we mounted training and test data
- logs: contains logged model training output
- model: trained model artifacts
- output: contains predictions. Also contains output from hyper-parameter tuning, if run. 

In [30]:
%%bash
ls -R ml_vol

ml_vol:
data
logs
model
output

ml_vol/data:
test
train

ml_vol/data/test:
ratings_test.csv

ml_vol/data/train:
attribute_defn.csv
ratings_train.csv
user_attributes.csv

ml_vol/logs:

ml_vol/model:
checkpoint
model_params
model_weights.data-00000-of-00001
model_weights.index
preprocess_pipe.save

ml_vol/output:
predictions.csv
scores.csv


# Push to docker hub 
Need permission to push to this repository on docker hub

In [ ]:
%%bash
docker push abudesai/rec_base_autorec:v1.0.0

# Stop Container and Remove Image

In [ ]:
%%bash
docker stop autorec

In [ ]:
%%bash
docker rm autorec

In [ ]:
%%bash
docker rmi abudesai/rec_base_autorec:v1.0.0